# env setup

In [109]:
import os
LANGCHAIN_API_KEY = os.getenv(key="LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv(key="LANGCHAIN_ENDPOINT")
LANGCHAIN_TRACING_V2 = os.getenv(key="LANGCHAIN_TRACING_V2")
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.environ['GROQ_API_KEY']

# Imports

In [110]:
from langchain_community.chat_models import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.pydantic_v1 import BaseModel

# define llms

In [111]:
phi_llm = ChatOllama(model='phi',temperature=0.1,timeout=300)
gemma_llm = ChatOllama(model='gemma:2b',temperature=0.1,timeout=300)
llm = phi_llm

# DB: Connect to a SQLite DB.

In [112]:
db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info= 0)

In [113]:
# query the db
query = "SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson'"
db.run(query)

"[('Golden State Warriors',)]"

In [114]:
query = "SELECT NAME FROM nba_roster WHERE AGE = 19"
db.run(query)

"[('Jordan Walsh',), ('Noah Clowney',), ('Dariq Whitehead',), ('Amari Bailey',), ('James Nnaji',), ('Nick Smith Jr.',), ('Julian Phillips',), ('Emoni Bates',), ('Dereck Lively II',), ('Jalen Duren',), ('Cam Whitmore',), ('Chris Livingston',), ('Jaylen Martin',), ('Cason Wallace',), ('Anthony Black',), ('Scoot Henderson',), ('Sidy Cissoko',), ('Victor Wembanyama',), ('Gradey Dick',), ('Keyonte George',), ('Taylor Hendricks',), ('Brice Sensabaugh',), ('Gradey Dick',), ('Bilal Coulibaly',)]"

In [115]:
def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

# prompts and templates

In [132]:
template = """Based on the table schema below, return a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query: """  # noqa: E501

# template = """Based on the table schema below, return only a SQL query that would answer the user's question. No pre-amble.:
# {schema}

# Question: {question}
# Answer:"""  # noqa: E501


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        MessagesPlaceholder(variable_name="history"),
        ("human", template),
    ]
)

In [133]:
print(prompt)

input_variables=['history', 'question', 'schema'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given an input question, convert it to a SQL query. No pre-amble.')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template="Based on the table schema below, return a SQL query that would answer the user's question:\n{schema}\n\nQuestion: {question}\nSQL Query: "))]


In [134]:
memory = ConversationBufferMemory(return_messages=True)

# Chains

In [138]:
# Chain to query with memory

sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    # | llm.bind(stop=["\nUser:"])
    # | gemma_llm.bind(stop=["\n"])
    # | llm.bind(stop=["\nUser:", "\nRules:", "\nAssistant"])
    # | llm
    | gemma_llm
    | StrOutputParser()
)

In [137]:
def format_query(query):
    return query.strip().replace("```sql", "").replace("```", "")

In [158]:
sql_chain_format = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    # | llm.bind(stop=["\nUser:"])
    # | gemma_llm.bind(stop=["\n"])
    # | llm.bind(stop=["\nUser:", "\nRules:", "\nAssistant"])
    # | llm
    | gemma_llm
    | StrOutputParser()
    | RunnableLambda(format_query)
)

In [159]:
sql_chain_format.invoke({"question": "What team is Klay Thompson on?"})

"\nSELECT Team\nFROM nba_roster\nWHERE NAME = 'Klay Thompson';\n"

In [144]:
formatted_chain = sql_chain | RunnableLambda(format_query)

In [145]:
formatted_chain.invoke({"question": "What team is Klay Thompson on?"})

"\nSELECT Team\nFROM nba_roster\nWHERE NAME = 'Klay Thompson';\n"

In [136]:
sql_chain.invoke({"question": "What team is Klay Thompson on?"})

"```sql\nSELECT Team\nFROM nba_roster\nWHERE NAME = 'Klay Thompson';\n```"

In [89]:
def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]

In [90]:
sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save

In [91]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""  # noqa: E501

prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural "
            "language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

In [92]:
prompt_response

ChatPromptTemplate(input_variables=['query', 'question', 'response', 'schema'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given an input question and SQL response, convert it to a natural language answer. No pre-amble.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query', 'question', 'response', 'schema'], template='Based on the table schema below, question, sql query, and sql response, write a natural language response:\n{schema}\n\nQuestion: {question}\nSQL Query: {query}\nSQL Response: {response}'))])

In [93]:
# Supply the input types to the prompt
class InputType(BaseModel):
    question: str

In [146]:
chain = (
    RunnablePassthrough.assign(query=formatted_chain).with_types(
        input_type=InputType
    )
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | gemma_llm
)

In [122]:
def format_query(query):
    return query.strip().replace("```sql", "").replace("```", "")


In [123]:
formatted_query = format_query("```sql\nSELECT * FROM table;\n```")
print(formatted_query)



SELECT * FROM table;



In [127]:
chain = (
    # RunnablePassthrough.assign({"query":sql_response_memory | format_query}).with_types(
    #     input_type=InputType
    # )
    {"query":sql_response_memory | format_query, "question": RunnablePassthrough()}
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | gemma_llm
)

In [100]:
# chain = (
#     RunnablePassthrough.assign(
#         query=lambda x: x.strip().replace("```sql", "").replace("```", "")
#     ).with_types(input_type=InputType)
#     | RunnablePassthrough.assign(
#         schema=get_schema,
#         response=lambda x: db.run(x["query"]),
#     )
#     | prompt_response
#     | gemma_llm
# )


In [151]:
sql_response_memory_formatted = RunnablePassthrough.assign(output=formatted_chain) | save

In [152]:
chain_format = (
    RunnablePassthrough.assign(query=sql_response_memory_formatted).with_types(
        input_type=InputType
    )
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | gemma_llm
)

In [153]:
chain_format.invoke({"question": "give me name of all players who's age is 19 years?"})

AIMessage(content='Based on the provided table schema and SQL query, the players who are 19 years old are:\n\n- Jordan Walsh\n- Noah Clowney\n- Dariq Whitehead\n- Amari Bailey\n- James Nnaji\n- Nick Smith Jr.\n- Julian Phillips\n- Emoni Bates\n- Dereck Lively II\n- Jalen Duren\n- Cam Whitmore\n- Chris Livingston\n- Jaylen Martin\n- Cason Wallace\n- Anthony Black\n- Scoot Henderson\n- Sidy Cissoko\n- Victor Wembanyama\n- Gradey Dick\n- Keyonte George\n- Taylor Hendricks\n- Brice Sensabaugh', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T17:49:17.6167819Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 8883895300, 'load_duration': 4004800, 'prompt_eval_count': 319, 'prompt_eval_duration': 4459606000, 'eval_count': 136, 'eval_duration': 4342150000}, id='run-7a74b21e-44c6-48bf-a54e-9009243a17d6-0')

In [147]:
chain.invoke({"question": "What team is Klay Thompson on?"})

AIMessage(content='Based on the provided table schema and SQL query, Klay Thompson is currently playing for the Golden State Warriors.', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T17:43:36.3455192Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 3735907100, 'load_duration': 3458700, 'prompt_eval_count': 168, 'prompt_eval_duration': 3097719000, 'eval_count': 23, 'eval_duration': 570565000}, id='run-f9e0ac77-5c9d-4233-81ab-3070765863bb-0')

In [148]:
chain.invoke({"question": "Give me total number of players in Golden State Warriors Team?"})

AIMessage(content='Based on the provided table schema and SQL response, there are 17 players on the Golden State Warriors team.', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T17:44:15.0543721Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 3642090300, 'load_duration': 2752300, 'prompt_eval_count': 169, 'prompt_eval_duration': 3028625000, 'eval_count': 24, 'eval_duration': 601283000}, id='run-6a11faa0-2a79-4f26-b34a-595fa64dee17-0')

In [149]:
chain.invoke({"question": "How many total different teams are there in database?"})

AIMessage(content='Based on the provided table schema and SQL response, there are 30 distinct teams in the NBA roster.', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T17:46:29.1650226Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 3730860700, 'load_duration': 3313100, 'prompt_eval_count': 169, 'prompt_eval_duration': 3085372000, 'eval_count': 23, 'eval_duration': 625059000}, id='run-a1f3f2d7-2e6b-4fc4-aa54-b995e39fced7-0')

In [72]:
chain.invoke({"question": "who is oldest player in nba roaster?"})

AIMessage(content='Based on the provided table schema and SQL response, the oldest player in the NBA roster is not specified. Therefore, I am unable to provide a natural language answer to this question.', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T10:37:55.4473969Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 3558546900, 'load_duration': 17776200, 'prompt_eval_count': 158, 'prompt_eval_duration': 2625240000, 'eval_count': 37, 'eval_duration': 905462000}, id='run-a4d3a68e-752b-48aa-90ff-344ed1d90d25-0')

In [73]:
chain.invoke({"question": "who is youngest player in nba roaster?"})

AIMessage(content='Based on the provided table schema and SQL response, the youngest player in the NBA roster is 18 years old.', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T10:40:18.6704592Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 3295206300, 'load_duration': 2795200, 'prompt_eval_count': 159, 'prompt_eval_duration': 2678057000, 'eval_count': 25, 'eval_duration': 594757000}, id='run-e36e9aa1-7377-4300-98cd-91d8141bb825-0')

In [74]:
chain.invoke({"question": "give me name of all players who's age is 19 years?"})

AIMessage(content='Based on the provided table schema and SQL response, the players who are 19 years old are:\n\n- Jordan Walsh\n- Noah Clowney\n- Dariq Whitehead\n- Amari Bailey\n- James Nnaji\n- Nick Smith Jr.\n- Julian Phillips\n- Emoni Bates\n- Dereck Lively II\n- Jalen Duren\n- Cam Whitmore\n- Chris Livingston\n- Jaylen Martin\n- Cason Wallace\n- Anthony Black\n- Scoot Henderson\n- Sidy Cissoko\n- Victor Wembanyama\n- Gradey Dick\n- Keyonte George\n- Taylor Hendricks\n- Brice Sensabaugh\n- Gradey Dick\n- Bilal Coulibaly', response_metadata={'model': 'gemma:2b', 'created_at': '2024-04-24T10:42:43.090322Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 9137884400, 'load_duration': 3725500, 'prompt_eval_count': 315, 'prompt_eval_duration': 4424181000, 'eval_count': 148, 'eval_duration': 4659951000}, id='run-cfe27e6c-2467-466e-b5ac-d4a72d7d8322-0')